In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature as skft
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, SVR, LinearSVC
import cv2

In [2]:
import numpy
import math
def makeRFSFilters():
    SUP = 49
    SCALES = [1,2,4]
    NORIENT = 6
    
    NROTINV = 2
    NBAR = len(SCALES) * NORIENT 
    NEDGE = len(SCALES) * NORIENT
    NF = NEDGE + NBAR + NROTINV
    F = numpy.zeros((NF, SUP, SUP))
    hsup = 24
    nownum = 0
    x = []
    y = []
    for i in range(-hsup, hsup+1):
        for j in range(-hsup, hsup+1):
            x.append(j)
            y.append(i)
            nownum += 1
    orgpts = numpy.zeros((2, SUP*SUP))
    for i in range(SUP*SUP):
        orgpts[0][i] = x[i]
        orgpts[1][i] = y[i]
    
    count = 0
    for scale in range(len(SCALES)):
        for orient in range(NORIENT):
            angle = math.pi * orient / NORIENT
            Turn = numpy.zeros((2, 2))
            c = math.cos(angle)
            s = math.sin(angle)
            Turn[1, 1] = c
            Turn[0, 0] = c
            Turn[0, 1] = -s
            Turn[1, 0] = s
            Turn = numpy.mat(Turn)
            orgpts = numpy.mat(orgpts)
            rotpts = Turn * orgpts
            F[count, :, :] = makefilter(SCALES[scale], 0, 1, rotpts, SUP)
            F[count+NEDGE, :, :] = makefilter(SCALES[scale], 0, 2, rotpts, SUP)
            count = count + 1
            
    F[NBAR+NEDGE, :, :] = normalize(imgGaussian(hsup, 10))
    F[NBAR+NEDGE+1, :, :] = normalize(imgLaplacian(hsup, 10))
    return F

def makefilter(scale, phasex, phasey, pts, sup):
    gx = gaussld(3*scale, 0, pts[0, :], phasex)
    gy = gaussld(1*scale, 0, pts[1, :], phasey)
    f = normalize(numpy.multiply(gx, gy).reshape(sup, sup))
    return f

def gaussld(sigma, mean, x, ord):
    x = x - mean
    x = numpy.mat(x)
    num = numpy.multiply(x, x)
    variance = sigma ** 2
    denom = 2 * variance
    g = numpy.exp(-1.0*num / denom) / numpy.sqrt(numpy.pi * denom)
    if ord == 1:
        g = numpy.multiply(g, x/variance)
    elif ord == 2:
        g = numpy.multiply(g, (num - variance) / (variance**2))
    return g
    
def normalize(f):
    f = f - f.mean()
    f = f / abs(f).sum()
    return f

In [3]:
def imgGaussian(size, sigma):
    '''
    :param sigma: σ标准差
    :return: 高斯滤波器的模板
    '''
    img_h = img_w = size*2+1
    gaussian_mat = numpy.zeros((img_h, img_w))
    for x in range(-size, size + 1):
        for y in range(-size,size + 1):
            gaussian_mat[x + size][y + size] = numpy.exp(-0.5 * (x ** 2 + y ** 2) / (sigma ** 2))
    return gaussian_mat

def imgLaplacian(size, sigma):
    laplacian_mat = -imgGaussian(size, 10)
    laplacian_mat[size+1, size+1] = -(laplacian_mat.sum() - laplacian_mat[size+1, size+1])
    return laplacian_mat

In [4]:
F = makeRFSFilters()

In [5]:
classes = ['aluminium_foil', 'brown_bread', 'corduroy', 'cotton', 'cracker', 'linen', 'orange_peel', 'sandpaper', 'sponge', 'styrofoam']
sample_numbers = ['15', '48', '42', '46', '60', '44', '55', '06', '21', '20']
# 10 classes in total, 81 pics in each class which contains 9 pics and each pic has 9 scales.
# for each class, use 7 pics to train and 2 to test

In [6]:
def load_data():
    # trian:test = 8:1
    train_data = []
    test_data = []
    train_label = []
    test_label = []
    for c in range(len(classes)):
        for p in range(1, 10):  # 9 different pics actually
            for s in range(1, 10):  # each pic has 9 scales
                
                if is_grey:
                    path = './KTH-TIPS/kth_tips_grey_200x200/KTH_TIPS/' + classes[c] + '/' + sample_numbers[c] + '-scale_' + str(s) + '_im_' + str(p) + '_grey.png'
                else:
                    path = './KTH-TIPS/kth_tips_col_200x200/KTH_TIPS/' + classes[c] + '/' + sample_numbers[c] + '-scale_' + str(s) + '_im_' + str(p) + '_col.png'
                
                img = plt.imread(path)
                img = cv2.resize(img, (200,200), interpolation=cv2.INTER_AREA) 
                if p < 9:
                    train_data.append(img)
                    train_label.append(c)
                else:
                    test_data.append(img)
                    test_label.append(c)
                    
    return train_data, test_data, np.array(train_label), np.array(test_label)

In [7]:
radius = 1
n_point = radius * 8
def texture_detect_grey(train_data, test_data):
    train_hist = np.zeros( (len(train_data), 256) )
    test_hist = np.zeros( (len(test_data), 256) )
    for i in range(len(train_data)):
        lbp = skft.local_binary_pattern(train_data[i], n_point, radius, 'default')
        max_bins = int(lbp.max() + 1)
        train_hist[i], _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))
    for i in range(len(test_data)):
        lbp = skft.local_binary_pattern(test_data[i], n_point, radius, 'default')
        max_bins = int(lbp.max() + 1)
        test_hist[i], _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))
    return train_hist, test_hist


def texture_detect_color(train_data, test_data):
    train_hist = np.zeros((len(train_data),256*3))
    test_hist = np.zeros((len(test_data),256*3))
    for i in np.arange(len(train_data)):
        lbp_r=skft.local_binary_pattern(train_data[i][:, :, 0], n_point, radius, 'default')
        lbp_g=skft.local_binary_pattern(train_data[i][:, :, 1], n_point, radius, 'default')
        lbp_b=skft.local_binary_pattern(train_data[i][:, :, 2], n_point, radius, 'default')
        max_bins = int(lbp_r.max() + 1)
        train_hist[i][:256], _ = np.histogram(lbp_r, normed=True, bins=max_bins, range=(0, max_bins))
        train_hist[i][256:512], _ = np.histogram(lbp_g, normed=True, bins=max_bins, range=(0, max_bins))
        train_hist[i][512:], _ = np.histogram(lbp_b, normed=True, bins=max_bins, range=(0, max_bins))
    for i in np.arange(len(test_data)):
        lbp_r=skft.local_binary_pattern(test_data[i][:, :, 0], n_point, radius, 'default')
        lbp_g=skft.local_binary_pattern(test_data[i][:, :, 1], n_point, radius, 'default')
        lbp_b=skft.local_binary_pattern(test_data[i][:, :, 2], n_point, radius, 'default')
        max_bins = int(lbp_r.max() + 1)
        test_hist[i][:256], _ = np.histogram(lbp_r, normed=True, bins=max_bins, range=(0, max_bins))
        test_hist[i][256:512], _ = np.histogram(lbp_g, normed=True, bins=max_bins, range=(0, max_bins))
        test_hist[i][512:], _ = np.histogram(lbp_b, normed=True, bins=max_bins, range=(0, max_bins))
    return train_hist, test_hist

In [8]:
# max pooling
def max_pooling(img, G=8):
    # Max Pooling
    
    size = img.shape
    if(len(size)==2):
        H, W = img.shape
        Nh = int(H / G)
        Nw = int(W / G)
        out = np.zeros((Nh,Nw))
        for y in range(Nh):
            for x in range(Nw):
                    out[y][x] = np.max(img[G*y:G*(y+1), G*x:G*(x+1)])
    else:
        H, W, C = img.shape
        Nh = int(H / G)
        Nw = int(W / G)
        out = np.zeros((Nh,Nw,C))
        for y in range(Nh):
            for x in range(Nw):
                for c in range(C):
                    out[y][x][c] = np.max(img[G*y:G*(y+1), G*x:G*(x+1), c])

    return out

In [9]:
def conv_block(img, kernal, pooling_size=8):
    
    img = cv2.resize(img, (200,200), interpolation=cv2.INTER_AREA) 
    all_feature = []
    for item in kernal:
        conv = cv2.filter2D(img, -1, item)
        conv = max_pooling(conv, pooling_size)
        conv = conv.flatten().tolist()
        all_feature.append(conv)
        
    all_feature = np.array(all_feature).flatten().tolist()
    
    return all_feature

In [10]:
def run():
    
    train_data, test_data, train_label, test_label = load_data()
    
    if is_lbp:
        if is_grey:
            train_hist, test_hist = texture_detect_grey(train_data, test_data)
        else:
            train_hist, test_hist = texture_detect_color(train_data, test_data)
    else:
        train_hist = np.array([conv_block(item, F, 25) for item in train_data])
        test_hist = np.array([conv_block(item, F, 25) for item in test_data])
        
        
    model = OneVsRestClassifier(SVR(kernel='linear', C=1e3, gamma=0.1))
    model = model.fit(train_hist, train_label)
    acc = model.score(test_hist, test_label)
    result = model.predict(test_hist)
    
    print(acc)
    print(result)

In [11]:
is_grey = True
is_lbp = False

run()

0.8
[8 3 2 3 3 3 7 7 7 1 1 7 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 8 3 3 3 3 3 3 3 3 1
 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 9 9 7 7 7 7 7 7 8 8
 8 8 8 8 8 8 8 9 9 7 9 9 7 9 9 9]


In [12]:
is_grey = False
is_lbp = False

run()

0.8777777777777778
[8 3 2 0 3 0 7 7 7 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 7 9 7 7 9 7 7 7 7 8 8
 8 8 8 8 8 8 8 9 9 7 9 9 9 9 9 9]


In [13]:
is_grey = True
is_lbp = True

run()

<ipython-input-7-ec5108f4377d>:9: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  train_hist[i], _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))
<ipython-input-7-ec5108f4377d>:13: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  test_hist[i], _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))


0.8
[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 6 0
 1 2 0 0 0 0 4 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 1 1 9 7 7 7 7 7 9 6 9
 9 8 8 8 8 1 8 4 9 9 9 9 9 9 9 9]


In [14]:
is_grey = False
is_lbp = True

run()

<ipython-input-7-ec5108f4377d>:25: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  train_hist[i][:256], _ = np.histogram(lbp_r, normed=True, bins=max_bins, range=(0, max_bins))
<ipython-input-7-ec5108f4377d>:26: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  train_hist[i][256:512], _ = np.histogram(lbp_g, normed=True, bins=max_bins, range=(0, max_bins))
<ipython-input-7-ec5108f4377d>:27: Visi

0.9444444444444444
[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 2 7 7 7 9 7 7 9 9 8 8
 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9]
